In [63]:
#pip install ee

In [64]:
#pip install geemap

# Import Libraries

In [65]:
import ee
import geemap
import pandas as pd
import numpy as np 
from matplotlib import pyplot
from io import StringIO


#ee.Authenticate()  # Only needed for the first time
ee.Initialize()
Map= geemap.Map()

In [66]:
# Read the lake IDs and coordinates from Excel data
data_path = 'ALTM-50-stations.xlsx'
df_lake_info = pd.read_excel(data_path, sheet_name='updated station coordinates', usecols=['SITE_ID', 'SITE_NAME','LATDD', 'LONDD'])

# Updated target_lake_ids list with string representation
target_lake_ids = [
    '020058', '020059', '020138', '020188', '020197',
    '030171', '030172', '040186', '040210', '040576',
    '040704', '040707', '040826', '040850', '040852',
    '040874', '040887', '041007', '050215', '050649',
    '050669', '050706', '050707', '060182', '060315A',
    '070859', '1A1-017', '1A1-029', '1A1-052', '1A1-059',
    '1A1-071', '1A1-087', '1A1-089', '1A1-102', '1A1-103',
    '1A1-105', '1A1-106', '1A1-107', '1A1-109', '1A1-110',
    '1A1-111', '1A1-112', '1A2-028', '1A2-066', '1A2-076',
    '1A2-077', '1A2-078', '1A3-001', '1A3-048', '1A3-065'
]




In [67]:
# Lake/Pond Names with Assigned Lake ID:
# SITE_ID
# - 020058: Little Hope Pond           - 020059: Big Hope Pond
# - 020138: East Copperas Pond         - 020188: Sunday Pond
# - 020197: Sochia Pond                - 030171: Grass Pond (3)
# - 030172: Little Clear Pond          - 040186: Loon Hollow Pond
# - 040210: Willys Lake                - 040576: Woods Lake
# - 040704: Middle Settlement Lake     - 040707: Middle Branch Lake
# - 040826: Limekiln Lake              - 040850: Squaw Lake
# - 040852: Indian Lake                - 040874: Brook Trout Lake
# - 040887: Lost Pond                  - 041007: North Lake
# - 050215: Willis Lake                - 050649: Long Pond
# - 050669: Carry Pond                 - 050706: Lake Colden
# - 050707: Avalanche Lake             - 060182: Little Simon Pond
# - 060315A: Raquette Lake Reservoir   - 070859: G Lake
# - 1A1-017: Constable Pond            - 1A1-029: Middle Pond
# - 1A1-052: Arbutus Pond              - 1A1-059: Sagamore Lake
# - 1A1-071: Black Pond                - 1A1-087: Windfall Pond
# - 1A1-089: Queer Lake                - 1A1-102: Heart Lake
# - 1A1-103: Big Moose Lake            - 1A1-105: Cascade Lake
# - 1A1-106: Dart Lake                 - 1A1-107: Little Echo Pond
# - 1A1-109: Moss Lake                 - 1A1-110: Lake Rondaxe
# - 1A1-111: Squash Pond               - 1A1-112: West Pond
# - 1A2-028: Owen Pond                 - 1A2-066: Jockeybush Lake
# - 1A2-076: Barnes Lake               - 1A2-077: Clear Pond
# - 1A2-078: Otter Lake                - 1A3-001: Nate Pond
# - 1A3-048: Grass Pond                - 1A3-065: South Lake (East Branch)

In [68]:
# Function to create a five-sided polygon around the latitude and longitude
def create_lake_polygon(lat, lon):
    # Assuming each pixel is approximately 20 meters in size
    buffer_distance_meters = 3 * 20  
    
    # Convert buffer distance from meters to degrees (approximate conversion)
    buffer_distance_degrees = buffer_distance_meters / 111320.0
    
    # Define the coordinates of the five vertices of the polygon
    coordinates = [
        [lon, lat + buffer_distance_degrees],
        [lon + buffer_distance_degrees, lat + buffer_distance_degrees/2],
        [lon + buffer_distance_degrees/2, lat - buffer_distance_degrees/2],
        [lon - buffer_distance_degrees/2, lat - buffer_distance_degrees/2],
        [lon - buffer_distance_degrees, lat + buffer_distance_degrees/2]
    ]
    
    # Create the polygon geometry
    polygon = ee.Geometry.Polygon(coordinates)
    
    return polygon

In [69]:
sentinel2_bands =['B1','B2','B3','B4','B5','B6','B8','B8A','B11']
STD_NAMES = ['Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1','RedEdge2','RedEdge4','NIR','SWIR1']

# Masking Clouds

In [70]:
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Create the cloud mask
    cloudMask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Apply the cloud mask to the image
    maskedImage = image.updateMask(cloudMask)

    return maskedImage

In [71]:
# Function to calculate CDOM from Sentinel-2 images
def cdom(img, lake):
    cdo = img.expression("(20.3 - 10. * (b2 / b3) - 2.4 * (b3 / b4))", {
        'b1': img.select('Aerosols'),
        'b2': img.select('Blue'),
        'b3': img.select('Green'),
        'b4': img.select('Red')
    }).rename("CO")
    
    bad2 = cdo.where(cdo.gte(0), 1).rename("bad2")
    co = cdo.multiply(bad2).rename("CO")
    mask = co.neq(0)

    return img.addBands(co).clip(lake).updateMask(mask)

In [72]:
# Load Sentinel-2 data and apply the processing steps
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filter(ee.Filter.calendarRange(2019, 2023, 'year')) \
    .map(maskS2clouds) \
    .select(sentinel2_bands, STD_NAMES)\

# Function to process the data for a specific lake
def process_lake(lake_id, lat, lon):
    # Create a polygon around the lake using the latitude and longitude
    lake_polygon = create_lake_polygon(lat, lon)

    # Load the lake polygon to the map
    Map.addLayer(ee.Feature(lake_polygon), {'color': 'blue'}, f"Lake {lake_id}")

    # Load the lake-specific feature collection
    lake_collection = ee.FeatureCollection('projects/ee-touhedakhanom14/assets/lake_polygons') \
        .filter(ee.Filter.eq('SITE_ID', lake_id))

    # Apply the processing steps for the lake
    s2_lake = s2.filterBounds(lake_polygon) \
               .map(lambda img: cdom(img, lake_polygon)) \
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    
    # Display the first image in the collection
    Map.addLayer(s2_lake.first(), {}, f"Lake {lake_id} Image")

    return s2_lake

In [73]:
# Get the list of lake IDs
lake_ids = df_lake_info['SITE_ID'].tolist()

In [75]:
# Create a geemap Map
Map = geemap.Map()

# Get the list of lake IDs
lake_ids = df_lake_info['SITE_ID'].tolist()

# Prompt the user to enter the ID of the lake they want to analyze
selected_lake_id = input("Enter the ID of the lake you want to analyze: ")

# Check if the selected lake ID is in the list of lake IDs
if selected_lake_id not in lake_ids and int(selected_lake_id) not in lake_ids:
    print("Lake ID not found. Please select a valid lake ID.")
else:
    # Get the information for the selected lake
    lake_info = df_lake_info[df_lake_info['SITE_ID'] == selected_lake_id].iloc[0]
    lat = lake_info['LATDD']
    lon = lake_info['LONDD']

    # Create a polygon around the selected lake using the latitude and longitude
    lake_polygon = create_lake_polygon(lat, lon)

    # Call the process_lake function to process the data for the selected lake
    s2_lake = process_lake(selected_lake_id, lat, lon)

    Map.centerObject(lake_polygon, 14)
    # Display the map with the lake ID as the image name
    #Map.addLayer(s2.first(), {}, f"Lake {selected_lake_id} Image")

# Display the map
#Map.addLayerControl()
Map

Enter the ID of the lake you want to analyze: 1A1-103


Map(center=[43.816982288513294, -74.85610999995801], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
#Map.user_roi.getInfo()

In [ ]:
#Map.addLayer(s2.first(), {}, lake_name + " Image")

In [ ]:
# Convert the image collection to a list and get the last image
s2_list = s2.toList(s2.size())
last_image = ee.Image(s2_list.get(-1))
#geemap.image_props(last_image).getInfo()

In [ ]:
#image = se.first()

In [ ]:
imageFirst = s2.first()
#geemap.image_props(imageFirst).getInfo()

In [ ]:
#print(s2.getInfo())

In [ ]:
vis_params = {'bands': ['Red', 'Green', 'Blue'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}

In [ ]:
vis_params = {'bands': ['Red', 'Green', 'Blue'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}

In [ ]:
first_image = s2.first()  # Get the first image from the collection
band_names = first_image.bandNames().getInfo()
print("Band Names:", band_names)

In [ ]:
#How many images?
#print(s2.size().getInfo())


# CDOM

In [ ]:
def station_mean(img): # station_mean function with a single parameter img
    #mean of a specific band (CO) within a region 
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake_polygon, scale=20).get('CO')
    return img.set('date', img.date().format()).set('CDOM',mean)

station_reduced_imgs = s2_lake.map(station_mean)
nested_list = station_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','CDOM']).get('list')
df = pd.DataFrame(nested_list.getInfo(), columns=['date','CDOM'])

pd.set_option('display.max_rows', None)
print(df)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

# Format 'date' column to exclude the time
df['date'] = df['date'].dt.date
df

In [ ]:
# import pandas as pd

# # Filter out CDOM values greater than 20
# df = df[df['CDOM'] <= 20]


# df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
# Create a figure with subplots and set the dimensions to 15 x 7
fig, ax = plt.subplots(figsize=(15, 7))

# Create the plot by setting our dataframe to the data argument
sns.scatterplot(data=df, x='date', y='CDOM', ax=ax)

# Set the labels and title
ax.set_ylabel('CDOM', fontsize=20)
ax.set_xlabel('date', fontsize=20)
ax.set_title('Cdom Values over Time', fontsize=20)
ax.set_ylim(0,15)

# Format the x-axis tick labels to display only the year
# ax.xaxis.set_major_formatter(plt.FixedFormatter(df['date'].dt.year.unique()))

# Display the plot
plt.show()


# Reflectance Values

In [ ]:
# Function to calculate the mean of specific bands indicated by STD_NAMES for a given lake polygon
def reflectance(img, lake):
    reflectance_values = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake_polygon, scale=30).select(STD_NAMES)
    return ee.Feature(None, {'date': img.date().format(), 'reflectance': reflectance_values})

# Calculate the reflectance values for all dates for the selected lake
s2_reflectance = s2_lake.map(lambda img: reflectance(img, lake_polygon))

# Convert the image collection to a list and get the data for the DataFrame
s2_list = s2_reflectance.reduceColumns(ee.Reducer.toList(2), ['date', 'reflectance']).values().get(0)
df_s2_reflectance = pd.DataFrame(s2_list.getInfo(), columns=['date', 'reflectance'])

# Convert date column to datetime and extract date only
df_s2_reflectance['date'] = pd.to_datetime(df_s2_reflectance['date']).dt.date

# Set the reflectance column values to a dictionary
df_s2_reflectance['reflectance'] = df_s2_reflectance['reflectance'].apply(lambda x: {k: v for k, v in x.items() if v is not None})
df_s2_reflectance

In [ ]:
# Create empty lists to store the data
data = []

# Extract the bands, reflectance values, and dates where available
for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    if reflectance:
        for band in STD_NAMES:
            value = reflectance.get(band)
            data.append({'Band': band, 'Reflectance': value, 'Date': date})

# Create a dataframe from the data
df_data = pd.DataFrame(data)

# Display the dataframe
print(df_data)



In [ ]:
#Average band reflectance values for each month
data_by_month = {}

# Extract the bands, reflectance values, and dates where available
for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    
    if year == 2019:
        month = date.month
        
        if month not in data_by_month:
            data_by_month[month] = {'reflectance_values': {band: [] for band in STD_NAMES}}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month]['reflectance_values'][band].append(value)

# Calculate the average reflectance values for each band within each month
average_data = {'band': [], 'average_reflectance': []}

for month, data in data_by_month.items():
    for band, reflectance_values in data['reflectance_values'].items():
        if reflectance_values:  # Check if reflectance_values is not empty
            average_value = sum(reflectance_values) / len(reflectance_values)
            average_data['band'].append(band)
            average_data['average_reflectance'].append(average_value)

# Create a dataframe from the average data
df_average_data = pd.DataFrame(average_data)

# Display the dataframex
print(df_average_data)


In [ ]:
data_by_month = {}

for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    
    if year == 2019:
        month = date.month
        
        if month not in data_by_month:
            data_by_month[month] = {'band': [], 'dates': [], 'reflectance': []}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month]['band'].append(band)
                    data_by_month[month]['dates'].append(date)
                    data_by_month[month]['reflectance'].append(value)

plt.figure(figsize=(10, 6))

line_styles = ['-', '--', '-.', ':']  # List of line styles
num_styles = len(line_styles)
style_index = 0

for month, data in data_by_month.items():
    unique_dates = list(set(data['dates']))  # Get unique dates for the month
    line_style = line_styles[style_index % num_styles]  # Cycle through line styles
    
    average_reflectance_values = []
    for band in STD_NAMES:
        reflectance_values = [data['reflectance'][i] for i in range(len(data['band'])) if data['band'][i] == band]
        
        if len(reflectance_values) > 0:
            average_reflectance = sum(reflectance_values) / len(reflectance_values)
            average_reflectance_values.append(average_reflectance)
        else:
            average_reflectance_values.append(0)
    
    plt.plot(STD_NAMES, average_reflectance_values, linestyle=line_style, marker='o', label=month)
    style_index += 1

plt.xlabel('Band')
plt.ylabel('Average Reflectance')
plt.ylim(0, 1)  # Adjust the y-axis limit based on the reflectance range
plt.title('Average Lake Surface Reflectance VS Bands for 2019 (All Months)')
plt.legend()
plt.show()




In [ ]:
import numpy as np

data_by_month = {}

# Iterate over the dataframe rows
for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    
    # Filter data for specific months
    if year == 2019 and date.month in [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
        month = date.month
        
        if month not in data_by_month:
            data_by_month[month] = {band: [] for band in STD_NAMES}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month][band].append(value)
                else:
                    data_by_month[month][band].append(np.nan)

# Calculate the average reflectance values for each band within each month
averages_by_month = {
    month: {band: np.nanmean(values) for band, values in data.items()}
    for month, data in data_by_month.items()
}

# Define plot parameters
line_styles = ['-', '--', '-.', ':']  # List of line styles
num_styles = len(line_styles)

# Calculate the number of rows and columns for the subplots
num_months = len(data_by_month)
num_cols = 2  # Number of subplots per row
num_rows = (num_months + num_cols - 1) // num_cols

fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(30, 5 * num_rows))

# Flatten the axs array to iterate over subplots
axs = axs.flatten()

# Iterate over the data for each month
for i, (month, data) in enumerate(averages_by_month.items()):
    line_style = line_styles[i % num_styles]  # Cycle through line styles
    
    axs[i].set_xlabel('Band')
    axs[i].set_ylabel('Reflectance')
    axs[i].set_title(f'Average Lake Surface Reflectance VS Bands for {month}-2019')
    
    bands = list(data.keys())
    reflectance_values = list(data.values())
    axs[i].plot(bands, reflectance_values, linestyle=line_style, marker='o', label=f'{month}-2019')
    
    axs[i].legend()

plt.tight_layout()
plt.show()





In [ ]:
data_by_month = {}

for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    month = date.month
    
    if year == 2019 and month in [9, 10]:  # Filter for September (9) and October (10) only
        if month not in data_by_month:
            data_by_month[month] = {'band': [], 'reflectance': []}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month]['band'].append(band)
                    data_by_month[month]['reflectance'].append(value)

plt.figure(figsize=(10, 6))

line_styles = ['-', '--']  # List of line styles for two months
style_index = 0

for month, data in data_by_month.items():
    band_averages = {band: sum(data['reflectance'][i] for i, b in enumerate(data['band']) if b == band) / data['band'].count(band) for band in STD_NAMES}
    bands = list(band_averages.keys())
    reflectance_values = list(band_averages.values())
    
    line_style = line_styles[style_index]  # Cycle through line styles
    
    plt.plot(bands, reflectance_values, linestyle=line_style, marker='o', label=f'{month}')
    
    style_index += 1

plt.xlabel('Band')
plt.ylabel('Average Reflectance')
plt.ylim(0, 0.2)  # Adjust the y-axis limit based on the reflectance range
plt.title('Average Lake Surface Reflectance VS Bands for September and October 2019')
plt.legend(title='Months')
plt.show()

# Display the data table for September and October
for month, data in data_by_month.items():
    print(f"Data table for Month {month}:")
    df = pd.DataFrame(data)
    print(df)
    print()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have already loaded the data into df_s2_reflectance and defined STD_NAMES

data_by_month = {}

for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    month = date.month
    
    if year == 2019 and month == 9:  # Filter for September (month 9) only
        if month not in data_by_month:
            data_by_month[month] = {band: [] for band in STD_NAMES}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month][band].append(value)

averages_by_band = {
    band: sum(data) / len(data) for band, data in data_by_month[9].items()
}

plt.figure(figsize=(10, 6))

bands = list(averages_by_band.keys())
reflectance_values = list(averages_by_band.values())

plt.plot(bands, reflectance_values, linestyle='-', marker='o', label='9')

plt.xlabel('Band')
plt.ylabel('Average Reflectance')
plt.ylim(0, 0.2)  # Adjust the y-axis limit based on the reflectance range
plt.title('Average Lake Surface Reflectance VS Bands for September 2019')

plt.legend()
plt.show()





In [ ]:
data_by_month = {}
monthly_average_reflectance = {}  # New dictionary to store average reflectance values

for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    
    if year == 2019:
        month = date.month
        
        if month in [9, 10]:  # Only consider data for September (9) and October (10)
            if month not in data_by_month:
                data_by_month[month] = {'band': [], 'dates': [], 'reflectance': []}
            
            if reflectance:
                for band in STD_NAMES:
                    value = reflectance.get(band)
                    if value is not None:
                        value /= 10000  # Divide by 10000 to convert from DN to reflectance
                        data_by_month[month]['band'].append(band)
                        data_by_month[month]['dates'].append(date)
                        data_by_month[month]['reflectance'].append(value)

# Calculate the average reflectance for each band for each month
for month, data in data_by_month.items():
    unique_bands = list(set(data['band']))
    average_reflectance = []
    for band in unique_bands:
        indices = [i for i, b in enumerate(data['band']) if b == band]
        reflectance_values = [data['reflectance'][i] for i in indices]
        avg_reflectance = sum(reflectance_values) / len(reflectance_values)
        average_reflectance.append(avg_reflectance)
    monthly_average_reflectance[month] = {'band': unique_bands, 'reflectance': average_reflectance}

plt.figure(figsize=(10, 6))

line_styles = ['-', '--', '-.', ':']  # List of line styles
num_styles = len(line_styles)
style_index = 0

for month, data in monthly_average_reflectance.items():
    line_style = line_styles[style_index % num_styles]  # Cycle through line styles
    plt.plot(data['band'], data['reflectance'], linestyle=line_style, marker='o', label=f"Month {month}")
    
    style_index += 1

plt.xlabel('Band')
plt.ylabel('Reflectance')
plt.ylim(0, 0.2)  # Adjust the y-axis limit based on the reflectance range
plt.title('Average Lake Surface Reflectance VS Bands for September and October 2019')
plt.legend()
plt.show()


Each Band Reflectance Over Time

In [ ]:
import pandas as pd

# Create empty lists to store the data
data = []

# Extract the bands, reflectance values, and dates where available
for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    if reflectance:
        for band in STD_NAMES:
            value = reflectance.get(band)
            data.append({'Band': band, 'Reflectance': value, 'Date': date})

# Create a dataframe from the data
df_data = pd.DataFrame(data)

# Display the dataframe
print(df_data)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

# Loop through each band and create a separate plot for each
for band in bands:
    df_band = df_data[df_data['Band'] == band]
    plt.figure(figsize=(6, 3))  # Adjust the figsize to make the plot smaller
    plt.plot(df_band['Date'], df_band['Reflectance'])
    plt.xlabel('Date')
    plt.ylabel('Reflectance')
    plt.title(f'{band} Reflectance Over Time')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

plt.figure(figsize=(10, 6))  # Adjust the figsize to make the plot larger
for band in bands:
    df_band = df_data[df_data['Band'] == band]
    plt.plot(df_band['Date'], df_band['Reflectance'], marker='o', label=band)

plt.xlabel('Date')
plt.ylabel('Reflectance')
plt.title('Reflectance Over Time for All Bands')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(10, 6))  # Adjust the figsize to make the plot larger
for band in bands:
    df_band = df_data[df_data['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    plt.plot(df_band['Date'], normalized_reflectance, marker='o', label=band)

plt.xlabel('Date')
plt.ylabel('Normalized Reflectance')
plt.title('Normalized Reflectance Over Time for All Bands')
plt.grid(True)
plt.ylim(0,0.4)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(12, 8))  # Adjust the figsize to make the plot larger
line_styles = ['-', '--', '-.', ':']  # Different line styles for each band

for idx, band in enumerate(bands):
    df_band = df_data[df_data['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    plt.plot(df_band['Date'], normalized_reflectance, marker='o', label=band, linestyle=line_styles[idx % len(line_styles)])

plt.xlabel('Date', fontsize=16)  # Increase text size for x-axis label
plt.ylabel('Normalized Reflectance', fontsize=16)  # Increase text size for y-axis label
plt.title('Normalized Reflectance Over Time for All Bands', fontsize=16)  # Increase text size for title
plt.ylim(0, 0.4)
plt.xticks(rotation=45)
plt.legend(prop={'size': 12}, frameon=True, edgecolor='black', fancybox=False, borderpad=1, borderaxespad=1, handlelength=2.5)  # Make the border bold
plt.tight_layout()
plt.grid(False)  # Remove the grid
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Filter data for the year 2019
df_data_2019 = df_data[df_data['Date'].dt.year == 2019]

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed
for band in df_data_2019['Band'].unique():
    df_band = df_data_2019[df_data_2019['Band'] == band]
    plt.plot(df_band['Date'], df_band['Reflectance'], label=f'{band}')

plt.xlabel('Date')
plt.ylabel('Reflectance')
plt.title('Reflectance Over Time (Year 2019)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Filter data for the year 2016
df_data_2019 = df_data[df_data['Date'].dt.year == 2019]

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed

# Define a list of different line styles for each band
line_styles = ['-', '--', '-.', ':']

# Get unique bands in the data
unique_bands = df_data_2019['Band'].unique()

# Plot each band with a different line style
for idx, band in enumerate(unique_bands):
    df_band = df_data_2019[df_data_2019['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    plt.plot(df_band['Date'], normalized_reflectance, label=f'{band}', linestyle=line_styles[idx % len(line_styles)])

plt.xlabel('Date')
plt.ylabel('Normalized Reflectance')
plt.title('Normalized Reflectance Over Time (Year 2019)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Filter data for the year 2016
df_data_2023 = df_data[df_data['Date'].dt.year == 2023]

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed
for band in df_data_2023['Band'].unique():
    df_band = df_data_2023[df_data_2023['Band'] == band]
    plt.plot(df_band['Date'], df_band['Reflectance'], label=f'{band}')

plt.xlabel('Date')
plt.ylabel('Reflectance')
plt.title('Reflectance Over Time (Year 2023)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Filter data for the year 2016
df_data_2023 = df_data[df_data['Date'].dt.year == 2023]

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed

# Define a list of different line styles for each band
line_styles = ['-', '--', '-.', ':']

# Get unique bands in the data
unique_bands = df_data_2023['Band'].unique()

# Plot each band with a different line style
for idx, band in enumerate(unique_bands):
    df_band = df_data_2023[df_data_2023['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    plt.plot(df_band['Date'], normalized_reflectance, label=f'{band}', linestyle=line_styles[idx % len(line_styles)])

plt.xlabel('Date')
plt.ylabel('Normalized Reflectance')
plt.title('Normalized Reflectance Over Time (Year 2023)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# DOC

In [ ]:
import pandas as pd

# Read the "Site Information" Excel sheet to get the mapping between lake names and SITE_ID
site_info_df = pd.read_excel('Site_Information_2022_8_1 (2).xlsx')

# Search for the corresponding SITE_ID for the selected lake
site_id = site_info_df.loc[site_info_df['SITE_ID'] == selected_lake_id, 'SITE_NAME'].iloc[0]


In [ ]:
import pandas as pd

# Read the data from the Excel file containing DOC values
data_doc = pd.read_excel('/Users/mindsmatter/Desktop/Lakes/LTM_Data_2023_3_9.xlsx', usecols=['DOC_MG_L', 'PROGRAM_ID','SITE_ID', 'DATE_SMP'])

data_doc = data_doc[data_doc['DOC_MG_L'].notnull()]

# Sort the DataFrame by 'DATE_SMP' column in ascending order
data_doc = data_doc.sort_values(by='DATE_SMP')

# Filter data from 2019 onwards and based on the obtained SITE_ID
data = data_doc[(data_doc['DATE_SMP'] >= '2019-01-01') & (data_doc['SITE_ID'] == selected_lake_id)]

# Rename columns for readability
data = data.rename(columns={'DATE_SMP': 'Date', 'DOC_MG_L': 'DOC', 'SITE_ID': 'SITE_ID'})

# Print the DataFrame containing the DOC values for the selected lake
print(data)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Create a figure with subplots and set the dimensions to 15 x 7
fig, ax = plt.subplots(figsize=(15, 7))

# Create the plot by setting our dataframe to the data argument
sns.scatterplot(data=data, x='Date', y='DOC', ax=ax)

# Set the labels and title
ax.set_ylabel('DOC', fontsize=20)
ax.set_xlabel('Date', fontsize=20)
ax.set_title('DOC Values over Time', fontsize=20)
ax.set_ylim(0, 10)

# Format the x-axis tick labels to display only the year
# ax.xaxis.set_major_formatter(plt.FixedFormatter(df['date'].dt.year.unique()))
#
# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), sharex=True)

# Seaborn scatter plot
sns.scatterplot(data=data, x='Date', y='DOC', ax=ax1)
ax1.set_xlabel('DATE')
ax1.set_ylabel('DOC')
ax1.set_ylim(0,15)

ax1.set_title('DOC Values Over Time')

# Seaborn scatter plot
sns.scatterplot(data=df, x='date', y='CDOM', ax=ax2)
ax2.set_xlabel('DATE')
ax2.set_ylabel('CDOM')
ax2.set_ylim(0,15)
ax2.set_title('CDOM Values Over Time')

# Adjust the spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# CDOM vs DOC Timeseries
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and an Axes object
fig, ax = plt.subplots(figsize=(15, 7))

# Seaborn scatter plot for CDOM
sns.scatterplot(data=df, x='date', y='CDOM', ax=ax, label='CDOM', color='blue')

# Create a twin Axes object for DOC
ax2 = ax.twinx()

# Matplotlib scatter plot for DOC
ax2.scatter(data['Date'], data['DOC'], label='DOC', color='red')

# Set the labels and title
ax.set_xlabel('DATE')
ax.set_ylabel('CDOM', color='blue')
ax2.set_ylabel('DOC', color='red')
ax.set_title('CDOM vs DOC')

# Set different ranges for the y-axes
ax.set_ylim(0,15)  # Adjust the range for CDOM
ax2.set_ylim(0, 15)  # Adjust the range for DOC

# Add legends
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.show()

In [ ]:
import pandas as pd

# Sort the 'data' DataFrame by date in ascending order
data = data.sort_values(by='Date')

# Sort the 'df' DataFrame by the 'date' column in ascending order
df.sort_values('date', inplace=True)

# Convert the 'date' column in the 'df' DataFrame to datetime dtype
df['date'] = pd.to_datetime(df['date'])

# Perform an inner merge with a five-day window tolerance
merged_inner = pd.merge_asof(df, data, left_on='date', right_on='Date', tolerance=pd.Timedelta(days=5))

# Get rid of the null values resulting from the merge
merged_inner = merged_inner.dropna()

print(merged_inner)
#print(df.dtypes)
#print(data.dtypes)



In [ ]:
# Drop duplicate rows based on DOC column, keeping the first occurrence
merged_inner = merged_inner.drop_duplicates(subset=['DOC'])

# Print the updated DataFrame
#print(merged_inner)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.scatter(merged_inner['DOC'], merged_inner['CDOM'])
# Setting labels and title
plt.xlabel('DOC')
plt.ylabel('CDOM')
plt.title(selected_lake_id + ' ' + 'Scatter Plot of DOC and CDOM for 5 days (Over Station)')
plt.ylim(0,15)
# Displaying the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and an Axes object
fig, ax = plt.subplots(figsize=(15, 7))

# Seaborn scatter plot for CDOM
sns.scatterplot(data=merged_inner, x='date', y='CDOM', ax=ax, label='CDOM', color='blue')

# Create a twin Axes object for DOC
ax2 = ax.twinx()

# Matplotlib scatter plot for DOC
ax2.scatter(merged_inner['date'], merged_inner['DOC'], label='DOC', color='red')

# Set the labels and title
ax.set_xlabel('DATE')
ax.set_ylabel('CDOM', color='blue')
ax2.set_ylabel('DOC', color='red')
ax.set_title('Merged CDOM vs DOC')

# Set different ranges for the y-axes
ax.set_ylim(0, 15)  # Adjust the range for CDOM
ax2.set_ylim(0, 15)  # Adjust the range for DOC

# Add legends
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Convert the date values to numeric format
merged_inner['date_numeric'] = pd.to_numeric(merged_inner['date']).astype(float)

# Create a figure and an Axes object
fig, ax = plt.subplots(figsize=(15, 7))

# Matplotlib scatter plot 
ax.scatter(merged_inner['date'], merged_inner['CDOM'], label='CDOM', color='blue')


# Create a twin Axes object for DOC
ax2 = ax.twinx()
ax2.scatter(merged_inner['date'], merged_inner['DOC'], label='DOC', color='red')


# Calculate the linear regression line for CDOM
cdom_x = merged_inner['date_numeric']
cdom_y = merged_inner['CDOM']
cdom_slope, cdom_intercept = np.polyfit(cdom_x, cdom_y, 1)
cdom_line = cdom_slope * cdom_x + cdom_intercept
#ax.plot(merged_inner['date'], cdom_line, color='blue', label='CDOM Line Fit')

# Calculate the linear regression line for DOC
doc_x = merged_inner['date_numeric']
doc_y = merged_inner['DOC']
doc_slope, doc_intercept = np.polyfit(doc_x, doc_y, 1)
doc_line = doc_slope * doc_x + doc_intercept
ax2.plot(merged_inner['date'], doc_line, color='red', label='DOC Line Fit')

# Set the labels and title
ax.set_xlabel('DATE')
ax.set_ylabel('CDOM', color='blue')
ax2.set_ylabel('DOC', color='red')
ax.set_title('Merged CDOM vs DOC for 5 days')

# Set different ranges for the y-axes
ax.set_ylim(4, 10)  # Adjust the range for CDOM
ax2.set_ylim(0, 10)  # Adjust the range for DOC

# Add legends
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.show()

In [ ]:
def station_mean(img, lake_polygon):
    # Mean of a specific band (CO) within a region
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake_polygon, scale=20).get('CO')
    return img.set('date', img.date().format()).set('CDOM', mean)

# Create an empty DataFrame to store CDOM values for all lakes
all_lakes_cdom = pd.DataFrame()

# Get CDOM values for all lakes
for lake_id in target_lake_ids:
    # Check if the lake ID exists in the DataFrame
    if lake_id in df_lake_info['SITE_ID'].values:
        # Get the information for the current lake
        lake_info = df_lake_info[df_lake_info['SITE_ID'] == lake_id].iloc[0]
        lat = lake_info['LATDD']
        lon = lake_info['LONDD']

        # Create a polygon around the current lake using the latitude and longitude
        lake_polygon = create_lake_polygon(lat, lon)

        # Load the lake-specific feature collection
        lake = ee.FeatureCollection('projects/ee-touhedakhanom14/assets/stations-coord')\
        .filter(ee.Filter.eq('SITE_NAME', lake_name))

        # Apply the processing steps for the current lake
        s2_lake = s2.filterBounds(lake_polygon) \
                   .map(lambda img: cdom(img, lake_polygon)) \
                   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25))

        # Map the station_mean function over the lake images
        station_reduced_imgs = s2_lake.map(lambda img: station_mean(img, lake_polygon))

        # Reduce the image collection to a nested list of dates and CDOM values
        nested_list = station_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date', 'CDOM']).get('list')

        # Convert the nested list to a DataFrame
        lake_cdom_df = pd.DataFrame(nested_list.getInfo(), columns=['date', 'CDOM'])

        # Add 'SITE_ID' and 'SITE_NAME' columns to the DataFrame
        lake_cdom_df['SITE_ID'] = lake_id
        lake_cdom_df['SITE_NAME'] = lake_info['SITE_NAME']

        # Append the DataFrame for the current lake to the overall DataFrame
        all_lakes_cdom = all_lakes_cdom.append(lake_cdom_df)

# Reset the DataFrame index
all_lakes_cdom.reset_index(drop=True, inplace=True)
# Convert the 'date' column to datetime format
all_lakes_cdom['date'] = pd.to_datetime(all_lakes_cdom['date'])

# Format 'date' column to exclude the time
all_lakes_cdom['date'] = all_lakes_cdom['date'].dt.date

# Print the DataFrame with CDOM values for all lakes
pd.set_option('display.max_rows', None)
print(all_lakes_cdom)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Convert the 'date' column to datetime format
all_lakes_cdom['date'] = pd.to_datetime(all_lakes_cdom['date'])

# Create a figure with subplots and set the dimensions to 15 x 7
fig, ax = plt.subplots(figsize=(15, 7))

# Create the plot by setting our DataFrame to the data argument
sns.scatterplot(data=all_lakes_cdom, x='date', y='CDOM', ax=ax)

# Set the labels and title
ax.set_ylabel('CDOM', fontsize=20)
ax.set_xlabel('Date', fontsize=20)
ax.set_title('CDOM Values over Time', fontsize=20)
ax.set_ylim(0, 20)

# Format the x-axis tick labels to display only the year
#ax.xaxis.set_major_formatter(plt.FixedFormatter(all_lakes_cdom['date'].dt.year.unique()))

# Rotate the x-axis tick labels for better readability
#plt.xticks(rotation=45)

# Display the plot
plt.show()


Reflectance Values

In [ ]:
import pandas as pd

# Function to calculate the mean of specific bands indicated by STD_NAMES for a given lake polygon
def reflectance(img, lake):
    reflectance_values = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake, scale=30).select(STD_NAMES)
    return ee.Feature(None, {'date': img.date().format(), 'reflectance': reflectance_values})

# Create an empty DataFrame to store reflectance values for all lakes
all_lakes_reflectance = pd.DataFrame()

# Get reflectance values for all lakes
for lake_id in target_lake_ids:
    # Check if the lake ID exists in the DataFrame
    if lake_id in df_lake_info['SITE_ID'].values:
        # Get the information for the current lake
        lake_info = df_lake_info[df_lake_info['SITE_ID'] == lake_id].iloc[0]
        lat = lake_info['LATDD']
        lon = lake_info['LONDD']

        # Create a polygon around the current lake using the latitude and longitude
        lake_polygon = create_lake_polygon(lat, lon)

        # Calculate the reflectance values for all dates for the current lake
        s2_reflectance = s2.filterBounds(lake_polygon).map(lambda img: reflectance(img, lake_polygon))

        # Convert the image collection to a list and get the data for the DataFrame
        s2_list = s2_reflectance.reduceColumns(ee.Reducer.toList(2), ['date', 'reflectance']).values().get(0)
        df_s2_reflectance = pd.DataFrame(s2_list.getInfo(), columns=['date', 'reflectance'])

        # Convert date column to datetime and extract date only
        df_s2_reflectance['date'] = pd.to_datetime(df_s2_reflectance['date']).dt.date

        # Set the reflectance column values to a dictionary
        df_s2_reflectance['reflectance'] = df_s2_reflectance['reflectance'].apply(lambda x: {k: v for k, v in x.items() if v is not None})

        # Add 'SITE_ID' and 'SITE_NAME' columns to the DataFrame
        df_s2_reflectance['SITE_ID'] = lake_id
        df_s2_reflectance['SITE_NAME'] = lake_info['SITE_NAME']

        # Append the DataFrame for the current lake to the overall DataFrame
        all_lakes_reflectance = all_lakes_reflectance.append(df_s2_reflectance)

# Reset the DataFrame index
all_lakes_reflectance.reset_index(drop=True, inplace=True)

# Print the DataFrame with reflectance values for all lakes
pd.set_option('display.max_rows', None)
print(all_lakes_reflectance)


In [ ]:
# Create empty lists to store the data
data = []

# Extract the bands, reflectance values, and dates where available
for index, row in all_lakes_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    if reflectance:
        for band in STD_NAMES:
            value = reflectance.get(band)
            data.append({'Band': band, 'Reflectance': value, 'Date': date})

# Create a DataFrame from the data
df_data = pd.DataFrame(data)

# Display the DataFrame with limited rows to avoid crashing the notebook
pd.set_option('display.max_rows', 20)  # Change the value as needed
print(df_data)


In [ ]:
# Average band reflectance values for each month
data_by_month = {}

# Extract the bands, reflectance values, and dates where available
for index, row in all_lakes_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    
    if year == 2019:
        month = date.month
        
        if month not in data_by_month:
            data_by_month[month] = {'reflectance_values': {band: [] for band in STD_NAMES}}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month]['reflectance_values'][band].append(value)

# Calculate the average reflectance values for each band within each month
average_data = {'band': [], 'average_reflectance': []}

for month, data in data_by_month.items():
    for band, reflectance_values in data['reflectance_values'].items():
        if reflectance_values:  # Check if reflectance_values is not empty
            average_value = sum(reflectance_values) / len(reflectance_values)
            average_data['band'].append(band)
            average_data['average_reflectance'].append(average_value)

# Create a DataFrame from the average data
df_average_data = pd.DataFrame(average_data)

# Display the DataFrame
print(df_average_data)

In [ ]:
data_by_month = {}

for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    
    if year == 2019:
        month = date.month
        
        if month not in data_by_month:
            data_by_month[month] = {'band': [], 'dates': [], 'reflectance': []}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month]['band'].append(band)
                    data_by_month[month]['dates'].append(date)
                    data_by_month[month]['reflectance'].append(value)

plt.figure(figsize=(10, 6))

line_styles = ['-', '--', '-.', ':']  # List of line styles
num_styles = len(line_styles)
style_index = 0

for month, data in data_by_month.items():
    unique_dates = list(set(data['dates']))  # Get unique dates for the month
    line_style = line_styles[style_index % num_styles]  # Cycle through line styles
    
    average_reflectance_values = []
    for band in STD_NAMES:
        reflectance_values = [data['reflectance'][i] for i in range(len(data['band'])) if data['band'][i] == band]
        
        if len(reflectance_values) > 0:
            average_reflectance = sum(reflectance_values) / len(reflectance_values)
            average_reflectance_values.append(average_reflectance)
        else:
            average_reflectance_values.append(0)
    
    plt.plot(STD_NAMES, average_reflectance_values, linestyle=line_style, marker='o', label=month)
    style_index += 1

plt.xlabel('Band')
plt.ylabel('Average Reflectance')
plt.ylim(0, 1)  # Adjust the y-axis limit based on the reflectance range
plt.title('Average Lake Surface Reflectance VS Bands for 2019 (All Months)')
plt.legend()
plt.show()


In [ ]:
data_by_month = {}

for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    
    if year == 2019:
        month = date.month
        
        if month not in data_by_month:
            data_by_month[month] = {'band': [], 'dates': [], 'reflectance': []}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month]['band'].append(band)
                    data_by_month[month]['dates'].append(date)
                    data_by_month[month]['reflectance'].append(value)

plt.figure(figsize=(10, 6))

line_styles = ['-', '--', '-.', ':']  # List of line styles
num_styles = len(line_styles)
style_index = 0

for month, data in data_by_month.items():
    unique_dates = list(set(data['dates']))  # Get unique dates for the month
    line_style = line_styles[style_index % num_styles]  # Cycle through line styles
    
    average_reflectance_values = []
    for band in STD_NAMES:
        reflectance_values = [data['reflectance'][i] for i in range(len(data['band'])) if data['band'][i] == band]
        
        if len(reflectance_values) > 0:
            average_reflectance = sum(reflectance_values) / len(reflectance_values)
            average_reflectance_values.append(average_reflectance)
        else:
            average_reflectance_values.append(0)
    
    plt.plot(STD_NAMES, average_reflectance_values, linestyle=line_style, marker='o', label=f"Month {month}")
    style_index += 1

plt.xlabel('Band')
plt.ylabel('Average Reflectance')
plt.ylim(0, 0.5)  # Adjust the y-axis limit based on the reflectance range
plt.title('Average Lake Surface Reflectance VS Bands for 2019 (All Months)')
plt.legend()
plt.show()


In [ ]:
data_by_month = {}

for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    year = date.year
    month = date.month
    
    if year == 2019 and month in [9, 10]:  # Filter for September (9) and October (10) only
        if month not in data_by_month:
            data_by_month[month] = {'band': [], 'reflectance': []}
        
        if reflectance:
            for band in STD_NAMES:
                value = reflectance.get(band)
                if value is not None:
                    value /= 10000  # Divide by 10000 to convert from DN to reflectance
                    data_by_month[month]['band'].append(band)
                    data_by_month[month]['reflectance'].append(value)

plt.figure(figsize=(10, 6))

line_styles = ['-', '--']  # List of line styles for two months
style_index = 0

for month, data in data_by_month.items():
    band_averages = {band: sum(data['reflectance'][i] for i, b in enumerate(data['band']) if b == band) / data['band'].count(band) for band in STD_NAMES}
    bands = list(band_averages.keys())
    reflectance_values = list(band_averages.values())
    
    line_style = line_styles[style_index]  # Cycle through line styles
    
    plt.plot(bands, reflectance_values, linestyle=line_style, marker='o', label=f'Month {month}')
    
    style_index += 1

plt.xlabel('Band')
plt.ylabel('Average Reflectance')
plt.ylim(0, 0.3)  # Adjust the y-axis limit based on the reflectance range
plt.title('Average Lake Surface Reflectance VS Bands for September and October 2019')
plt.legend()
plt.show()


In [ ]:
import pandas as pd

# Create empty lists to store the data
data = []

# Extract the bands, reflectance values, and dates where available
for index, row in df_s2_reflectance.iterrows():
    date = row['date']
    reflectance = row['reflectance']
    
    if reflectance:
        for band in STD_NAMES:
            value = reflectance.get(band)
            data.append({'Band': band, 'Reflectance': value, 'Date': date})

# Create a dataframe from the data
df_data = pd.DataFrame(data)

# Display the dataframe
print(df_data)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data_by_month['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data_by_month['Band'].unique()

# Loop through each band and create a separate plot for each
for band in bands:
    df_band = df_data_by_month[df_data_by_month['Band'] == band]
    plt.figure(figsize=(10, 6))  # Adjust the figsize to make the plot larger
    plt.plot(df_band['Date'], df_band['Reflectance'])
    plt.xlabel('Date')
    plt.ylabel('Reflectance')
    plt.title(f'{band} Reflectance Over Time for All Lakes')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

# Create a figure to display all bands in one graph
plt.figure(figsize=(10, 6))

# Loop through each band and plot the combined reflectance values
for band in bands:
    df_band = df_data[df_data['Band'] == band]
    plt.plot(df_band['Date'], df_band['Reflectance'], label=band)

plt.xlabel('Date')
plt.ylabel('Reflectance')
plt.title('Reflectance Over Time for All Lakes')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend(title='Band')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

plt.figure(figsize=(10, 6))  # Adjust the figsize to make the plot larger
for band in bands:
    df_band = df_data[df_data['Band'] == band]
    plt.plot(df_band['Date'], df_band['Reflectance'], marker='o', label=band)

plt.xlabel('Date')
plt.ylabel('Reflectance')
plt.title('Reflectance Over Time for All Bands')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(10, 6))  # Adjust the figsize to make the plot larger

# Create a separate line plot for each band
for band in bands:
    df_band = df_data[df_data['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    plt.plot(df_band['Date'], normalized_reflectance, label=band)

plt.xlabel('Date')
plt.ylabel('Normalized Reflectance')
plt.title('Normalized Reflectance Over Time for All Bands')
plt.grid(True)
plt.ylim(0, 1)  # Adjust the y-axis limit for better visualization
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Get a list of unique bands from the DataFrame
bands = df_data['Band'].unique()

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(10, 6))  # Adjust the figsize to make the plot larger

# Create an empty dictionary to store normalized reflectance for each band
normalized_data_by_band = {band: [] for band in bands}

# Loop through each band and calculate normalized reflectance for all lakes
for band in bands:
    df_band = df_data[df_data['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    normalized_data_by_band[band] = normalized_reflectance.tolist()

# Plot the normalized reflectance for each band for all lakes
for band in bands:
    plt.plot(df_data[df_data['Band'] == band]['Date'], normalized_data_by_band[band], marker='o', label=band)

plt.xlabel('Date')
plt.ylabel('Normalized Reflectance')
plt.title('Normalized Reflectance Over Time for All Bands and All Lakes')
plt.grid(True)
plt.ylim(0, 1)  # Adjust the y-axis limit for better visualization
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Filter data for the year 2019
df_data_2019 = df_data[df_data['Date'].dt.year == 2019]

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed
for band in df_data_2019['Band'].unique():
    df_band = df_data_2019[df_data_2019['Band'] == band]
    plt.plot(df_band['Date'], df_band['Reflectance'], label=f'{band}')

plt.xlabel('Date')
plt.ylabel('Reflectance')
plt.title('Reflectance Over Time (Year 2019)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Filter data for the year 2016
df_data_2019 = df_data[df_data['Date'].dt.year == 2019]

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed

# Define a list of different line styles for each band
line_styles = ['-', '--', '-.', ':']

# Get unique bands in the data
unique_bands = df_data_2019['Band'].unique()

# Plot each band with a different line style
for idx, band in enumerate(unique_bands):
    df_band = df_data_2019[df_data_2019['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    plt.plot(df_band['Date'], normalized_reflectance, label=f'{band}', linestyle=line_styles[idx % len(line_styles)])

plt.xlabel('Date')
plt.ylabel('Normalized Reflectance')
plt.title('Normalized Reflectance Over Time (Year 2019)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Divide reflectance values by 10000 to convert from DN to reflectance
df_data['Reflectance'] /= 10000

# Filter data for the year 2016
df_data_2023 = df_data[df_data['Date'].dt.year == 2023]

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed
for band in df_data_2023['Band'].unique():
    df_band = df_data_2023[df_data_2023['Band'] == band]
    plt.plot(df_band['Date'], df_band['Reflectance'], label=f'{band}')

plt.xlabel('Date')
plt.ylabel('Reflectance')
plt.title('Reflectance Over Time (Year 2023)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'Date' column to pandas datetime type
df_data['Date'] = pd.to_datetime(df_data['Date'])

# Filter data for the year 2016
df_data_2023 = df_data[df_data['Date'].dt.year == 2023]

# Min-Max normalization function
def min_max_normalize(data):
    return (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(8, 5))  # Adjust the figsize as needed

# Define a list of different line styles for each band
line_styles = ['-', '--', '-.', ':']

# Get unique bands in the data
unique_bands = df_data_2023['Band'].unique()

# Plot each band with a different line style
for idx, band in enumerate(unique_bands):
    df_band = df_data_2023[df_data_2023['Band'] == band]
    normalized_reflectance = min_max_normalize(df_band['Reflectance'])
    plt.plot(df_band['Date'], normalized_reflectance, label=f'{band}', linestyle=line_styles[idx % len(line_styles)])

plt.xlabel('Date')
plt.ylabel('Normalized Reflectance')
plt.title('Normalized Reflectance Over Time (Year 2023)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

DOC for all Lakes

In [ ]:
import pandas as pd

# Read the "Site Information" Excel sheet to get the mapping between lake names and SITE_ID
site_info_df = pd.read_excel('/Users/mindsmatter/Desktop/Lakes/Site_Information_2022_8_1 (2).xlsx')

# Read the data from the Excel file containing DOC values
data_doc = pd.read_excel('/Users/mindsmatter/Desktop/Lakes/LTM_Data_2023_3_9.xlsx', usecols=['DOC_MG_L', 'SITE_ID', 'DATE_SMP'])

data_doc = data_doc[data_doc['DOC_MG_L'].notnull()]

# Sort the DataFrame by 'DATE_SMP' column in ascending order
data_doc = data_doc.sort_values(by='DATE_SMP')

In [ ]:
import pandas as pd

# Create an empty DataFrame to store the results
all_lakes_doc_data = pd.DataFrame()

# Loop through each lake ID in the 'SITE_ID' column
for site_id in target_lake_ids:
    # Filter data for the current lake based on the obtained SITE_ID
    data = data_doc[data_doc['SITE_ID'] == site_id]
    # Rename columns for readability
    data = data.rename(columns={'DATE_SMP': 'Date', 'DOC_MG_L': 'DOC', 'SITE_ID': 'SITE_ID'})
    # Append the data for the current lake to the overall DataFrame
    all_lakes_doc_data = all_lakes_doc_data.append(data)

# Convert 'Date' column to pandas datetime type
all_lakes_doc_data['Date'] = pd.to_datetime(all_lakes_doc_data['Date'])

# Print the DataFrame containing the DOC values for all lakes
print(all_lakes_doc_data)



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Create an empty DataFrame to store the results
all_lakes_doc_data = pd.DataFrame()

# Loop through each lake ID in the 'SITE_ID' column
for site_id in data_doc['SITE_ID'].unique():
    # Filter data for the current lake based on the obtained SITE_ID
    data = data_doc[data_doc['SITE_ID'] == site_id]
    # Rename columns for readability
    data = data.rename(columns={'DATE_SMP': 'Date', 'DOC_MG_L': 'DOC', 'SITE_ID': 'SITE_ID'})
    # Append the data for the current lake to the overall DataFrame
    all_lakes_doc_data = all_lakes_doc_data.append(data)

# Convert 'Date' column to pandas datetime type
all_lakes_doc_data['Date'] = pd.to_datetime(all_lakes_doc_data['Date'])

# Create a figure with subplots and set the dimensions to 15 x 7
fig, ax = plt.subplots(figsize=(15, 7))

# Create the plot by setting our DataFrame to the data argument
sns.scatterplot(data=all_lakes_doc_data, x='Date', y='DOC', ax=ax)

# Set the labels and title
ax.set_ylabel('DOC', fontsize=20)
ax.set_xlabel('Date', fontsize=20)
ax.set_title('DOC Values over Time for All Lakes', fontsize=20)
ax.set_ylim(0, 25)

# Format the x-axis tick labels to display only the year
#ax.xaxis.set_major_formatter(plt.FixedFormatter(all_lakes_doc_data['Date'].dt.year.unique()))

# Display the plot
plt.show()



In [ ]:
#import seaborn as sns
#import matplotlib.pyplot as plt

## Convert 'Date' column to pandas datetime type
#df_data['Date'] = pd.to_datetime(df_data['Date'])

## Create a figure with two subplots side by side
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), sharex=True)

## Seaborn scatter plot for DOC data
#sns.scatterplot(data=df_data, x='Date', y='DOC', ax=ax1)
#ax1.set_xlabel('Date')
#ax1.set_ylabel('DOC')
#ax1.set_ylim(0, 15)
#ax1.set_title('DOC Values Over Time')

## Seaborn scatter plot for CDOM data
#sns.scatterplot(data=all_lakes_cdom, x='date', y='CDOM', ax=ax2)
#ax2.set_xlabel('Date')
#ax2.set_ylabel('CDOM')
#ax2.set_ylim(0, 15)
#ax2.set_title('CDOM Values Over Time')

## Adjust the spacing between subplots
#plt.tight_layout()

## Display the plot
#plt.show()


In [ ]:
#import seaborn as sns
#import matplotlib.pyplot as plt

## Create a figure and an Axes object
#fig, ax = plt.subplots(figsize=(15, 7))

## Seaborn scatter plot for CDOM
#sns.scatterplot(data=all_lakes_doc_data, x='Date', y='CDOM', ax=ax, label='CDOM', color='blue')

## Create a twin Axes object for DOC
#ax2 = ax.twinx()

## Matplotlib scatter plot for DOC
#ax2.scatter(all_lakes_doc_data['Date'], all_lakes_doc_data['DOC'], label='DOC', color='red')

## Set the labels and title
#ax.set_xlabel('DATE')
#ax.set_ylabel('CDOM', color='blue')
#ax2.set_ylabel('DOC', color='red')
#ax.set_title('CDOM vs DOC Timeseries for All Lakes')

## Set different ranges for the y-axes
#ax.set_ylim(0, 15)  # Adjust the range for CDOM
#ax2.set_ylim(0, 15)  # Adjust the range for DOC

## Add legends
#ax.legend(loc='upper left')
#ax2.legend(loc='upper right')

## Display the plot
#plt.show()


In [ ]:
import pandas as pd

# Sort the 'all_lakes_doc_data' DataFrame by 'Date' column in ascending order
all_lakes_doc_data = all_lakes_doc_data.sort_values(by='Date')

# Sort the 'all_lakes_cdom' DataFrame by the 'date' column in ascending order
all_lakes_cdom.sort_values('date', inplace=True)

# Convert the 'Date' column in the 'all_lakes_doc_data' DataFrame to datetime dtype
all_lakes_doc_data['Date'] = pd.to_datetime(all_lakes_doc_data['Date'])

# Convert the 'date' column in the 'all_lakes_cdom' DataFrame to datetime dtype
all_lakes_cdom['date'] = pd.to_datetime(all_lakes_cdom['date'])

# Perform an inner merge with a five-day window tolerance on 'DOC' and 'CDOM' data
merged_inner = pd.merge_asof(all_lakes_doc_data, all_lakes_cdom[['date', 'CDOM']], left_on='Date', right_on='date', tolerance=pd.Timedelta(days=5))

# Drop the redundant 'date' column from the merged DataFrame
merged_inner = merged_inner.drop(columns=['date'])

# Get rid of the null values resulting from the merge
merged_inner = merged_inner.dropna()

# Reset the DataFrame index
merged_inner.reset_index(drop=True, inplace=True)

# Convert the 'Date' column to datetime format
merged_inner['Date'] = pd.to_datetime(merged_inner['Date'])

# Format 'Date' column to exclude the time
merged_inner['Date'] = merged_inner['Date'].dt.date

# Print the DataFrame with merged DOC and CDOM values for all lakes
pd.set_option('display.max_rows', None)
print(merged_inner)



In [ ]:
# Drop duplicate rows based on DOC column, keeping the first occurrence
merged_inner = merged_inner.drop_duplicates(subset=['DOC'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Merge the 'SITE_NAME' column from the 'all_lakes_cdom' DataFrame into the merged DataFrame based on the 'SITE_ID' column
merged_inner = pd.merge(merged_inner, all_lakes_cdom[['SITE_ID', 'SITE_NAME']], on='SITE_ID', how='inner')

# Create a scatter plot with Seaborn, using 'SITE_NAME' as the hue
sns.scatterplot(data=merged_inner, x='DOC', y='CDOM', hue='SITE_NAME', palette='tab10')

# Setting labels and title
plt.xlabel('DOC')
plt.ylabel('CDOM')
plt.title('Scatter Plot of DOC and CDOM for 5 days (All Lakes)')
plt.ylim(0, 15)

# Displaying the legend outside the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Displaying the plot
plt.show()
